In [1]:
import random
from collections import defaultdict
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

# Path to the dataset
data_dir = '/kaggle/input/plantvillage-dataset/color'


data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the full dataset without transformations
full_dataset = datasets.ImageFolder(
    root=data_dir, 
    transform=data_transforms
    )

In [2]:
# Extract image paths and labels
image_paths = [sample[0] for sample in full_dataset.samples]
labels = [sample[1] for sample in full_dataset.samples]
class_names = full_dataset.classes
num_classes = len(class_names)


print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names}")

Number of classes: 38
Class names: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotte

In [3]:
# Create a DataLoader to iterate through the dataset
dataloader = DataLoader(full_dataset, batch_size=32, shuffle=False, num_workers=2)

# Initialize variables to accumulate pixel values
mean = 0.0
std = 0.0
total_images = 0

# Iterate through the dataset to compute mean and std
for images, _ in dataloader:
    batch_size = images.size(0)  # Number of images in the current batch
    images = images.view(batch_size, images.size(1), -1)  # Flatten height and width dimensions
    mean += images.mean(2).sum(0)  # Sum of means for each channel (RGB)
    std += images.std(2).sum(0)    # Sum of stds for each channel (RGB)
    total_images += batch_size     # Accumulate total number of images

# Divide by the total number of images to get the average mean and std
mean /= total_images
std /= total_images

print(f"Mean: {mean}")
print(f"Std: {std}")

Mean: tensor([0.4664, 0.4891, 0.4104])
Std: tensor([0.1761, 0.1500, 0.1925])


In [4]:
# Define the transformation pipeline
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),          # Resize images to 256x256 pixels
    transforms.ToTensor(),                  # Convert images to PyTorch tensors and scale to [0, 1]
    transforms.Normalize(mean=mean, std=std)  # Normalize using your dataset's mean and std
])

# Apply the transformations to the already loaded dataset
full_dataset.transform = data_transforms

In [5]:
# Manual stratified splitting
# Step 1: Organize indices by class
from collections import defaultdict

# Transform the list structure of labels to a dictionnary of lists representing keys and 
# indices representing values
class_to_indices = defaultdict(list)
for idx, label in enumerate(labels): #idx is the index in the dataset and label is the value at that position
    class_to_indices[label].append(idx)

# Step 2: Split indices for each class
train_indices = []
val_indices = []
test_indices = []

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

for label, indices in class_to_indices.items():
    # Shuffle the indices for this class
    random.shuffle(indices)

    # Detecting the number of samples in each class
    n_total = len(indices)
    n_train = int(train_ratio * n_total)
    n_val = int(val_ratio * n_total)
    n_test = n_total - n_train - n_val  # Ensure all samples are used

    # Split the indices
    train_idx = indices[:n_train]
    val_idx = indices[n_train:n_train + n_val]
    test_idx = indices[n_train + n_val:]

    # Append to the respective lists the samples of each class after splitting
    train_indices.extend(train_idx)
    val_indices.extend(val_idx)
    test_indices.extend(test_idx)

# Shuffle the final indices to ensure randomness across classes
random.shuffle(train_indices)
random.shuffle(val_indices)
random.shuffle(test_indices)

print(f"Total samples: {len(full_dataset)}")
print(f"Training samples: {len(train_indices)}")
print(f"Validation samples: {len(val_indices)}")
print(f"Testing samples: {len(test_indices)}")

Total samples: 54305
Training samples: 43429
Validation samples: 5417
Testing samples: 5459


In [6]:
from torch.utils.data import Subset

# Create subsets using the indices from the stratified split
train_subset = Subset(full_dataset, train_indices)
val_subset = Subset(full_dataset, val_indices)
test_subset = Subset(full_dataset, test_indices)

# Create DataLoaders
train_loader = DataLoader(
    train_subset,
    batch_size=32,
    shuffle=True,  # Shuffle for training
)

val_loader = DataLoader(
    val_subset,
    batch_size=32,
    shuffle=False,  # No shuffle for validation
)

test_loader = DataLoader(
    test_subset,
    batch_size=32,
    shuffle=False,  # No shuffle for testing
)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

class PlantDiseaseCNN(nn.Module):
    def __init__(self):
        super(PlantDiseaseCNN, self).__init__()
        
        # First conv uses a 7x7 kernel to quickly capture wide spatial context.
        # Then standard 3x3 blocks with pooling.
        self.conv1 = nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # Another pair of 3×3 blocks
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        # Another pooling
        self.pool2 = nn.MaxPool2d(2, 2)

        # Replace fully connected layer with GAP
        # GAP reduces each 32x32 feature map to 1x1, resulting in 128 outputs.
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, 39)  # Match the number of classes

    def forward(self, x):
        # Large kernel conv + stride
        x = nn.functional.relu(self.conv1(x))   # [batch, 32, 128, 128]
        
        # 2 small conv layers + pool
        x = nn.functional.relu(self.conv2(x))   # [batch, 64, 128, 128]
        x = nn.functional.relu(self.conv3(x))   # [batch, 64, 128, 128]
        x = self.pool(x)                        # -> [batch, 64, 64, 64]
        
        # Another 2 small conv + pool
        x = nn.functional.relu(self.conv4(x))   # [batch, 128, 64, 64]
        x = nn.functional.relu(self.conv5(x))   # [batch, 128, 64, 64]
        x = self.pool2(x)                       # -> [batch, 128, 32, 32]
        
        # Apply Global Average Pooling
        x = self.global_avg_pool(x)             # -> [batch, 128, 1, 1]
        x = x.view(x.size(0), -1)               # Flatten -> [batch, 128]
        
        # Final linear layer for classification
        x = self.fc(x)                          # -> [batch, 39]
        return x

In [8]:
model = PlantDiseaseCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move the model to GPU

num_epochs = 10
train_history = []
val_history = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        # Move data to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    train_loss = running_loss / len(train_loader)
    train_history.append((train_loss, train_accuracy))

    # Validation Loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Move data to GPU
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= len(val_loader)
    val_history.append((val_loss, val_accuracy))

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1/10
Train Loss: 3.6629, Train Accuracy: 3.57%
Validation Loss: 3.6607, Validation Accuracy: 6.20%
Epoch 2/10
Train Loss: 3.6586, Train Accuracy: 9.86%
Validation Loss: 3.6564, Validation Accuracy: 9.91%
Epoch 3/10
Train Loss: 3.6543, Train Accuracy: 9.87%
Validation Loss: 3.6522, Validation Accuracy: 9.88%
Epoch 4/10
Train Loss: 3.6500, Train Accuracy: 9.87%
Validation Loss: 3.6479, Validation Accuracy: 9.88%
Epoch 5/10
Train Loss: 3.6458, Train Accuracy: 9.87%
Validation Loss: 3.6437, Validation Accuracy: 9.88%
Epoch 6/10
Train Loss: 3.6415, Train Accuracy: 9.87%
Validation Loss: 3.6393, Validation Accuracy: 9.88%
Epoch 7/10
Train Loss: 3.6371, Train Accuracy: 9.87%
Validation Loss: 3.6349, Validation Accuracy: 9.88%
Epoch 8/10
Train Loss: 3.6325, Train Accuracy: 9.87%
Validation Loss: 3.6302, Validation Accuracy: 9.88%
Epoch 9/10
Train Loss: 3.6277, Train Accuracy: 9.87%
Validation Loss: 3.6253, Validation Accuracy: 9.88%
Epoch 10/10
Train Loss: 3.6226, Train Accuracy: 9.87%
V

In [10]:
# Evaluation
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        # Move inputs and labels to the same device as the model
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
test_loss /= len(val_loader)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Test Loss: 3.6199, Test Accuracy: 9.88%


In [11]:
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, precision_score

# Initialize variables to store results
all_labels = []
all_predictions = []

# Evaluation loop
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        # Move inputs and labels to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Predictions
        _, predicted = torch.max(outputs, 1)

        # Collect labels and predictions for metrics
        all_labels.extend(labels.cpu().numpy())  # Move to CPU for sklearn compatibility
        all_predictions.extend(predicted.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions, average="weighted")  # Weighted for multi-class
precision = precision_score(all_labels, all_predictions, average="weighted")  # Weighted for multi-class
f1 = f1_score(all_labels, all_predictions, average="weighted")  # Weighted for multi-class

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, target_names=class_names))


Accuracy: 9.84%
Precision: 0.0097
Recall: 0.0984
F1 Score: 0.0176

Classification Report:
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.00      0.00      0.00        63
                                 Apple___Black_rot       0.00      0.00      0.00        63
                          Apple___Cedar_apple_rust       0.00      0.00      0.00        28
                                   Apple___healthy       0.00      0.00      0.00       165
                               Blueberry___healthy       0.00      0.00      0.00       151
          Cherry_(including_sour)___Powdery_mildew       0.00      0.00      0.00       106
                 Cherry_(including_sour)___healthy       0.00      0.00      0.00        86
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.00      0.00      0.00        52
                       Corn_(maize)___Common_rust_       0.00      0.00      0.00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13